In [2]:
# https://github.com/onnx/tensorflow-onnx/blob/master/tutorials/huggingface-bert.ipynb
# https://github.com/microsoft/onnxruntime-inference-examples/blob/main/quantization/notebooks/bert/Bert-GLUE_OnnxRuntime_quantization.ipynb

# Converting a Huggingface model to ONNX with tf2onnx

This is a simple example how to convert a [huggingface](https://huggingface.co/) model to ONNX using [tf2onnx](https://github.com/onnx/tensorflow-onnx).

We use the [TFBertForQuestionAnswering](https://huggingface.co/transformers/model_doc/bert.html#tfbertforquestionanswering) example from huggingface.

Other models will work similar. You'll find additional examples for other models in our unit tests [here](https://github.com/onnx/tensorflow-onnx/blob/master/tests/huggingface.py).

## Install dependencies

In [3]:
!pip install tensorflow transformers tf2onnx onnxruntime

     |████████████████████████████████| 3.1 MB 5.5 MB/s 
     |████████████████████████████████| 435 kB 46.8 MB/s 
     |████████████████████████████████| 4.8 MB 31.7 MB/s 
     |████████████████████████████████| 3.3 MB 38.0 MB/s 
     |████████████████████████████████| 895 kB 30.8 MB/s 
     |████████████████████████████████| 61 kB 530 kB/s 
     |████████████████████████████████| 596 kB 42.8 MB/s 
     |████████████████████████████████| 12.7 MB 32.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 2.0
    Uninstalling flatbuffers-2.0:
      Successfully uninstalled flatbuffers-2.0


## The keras code

In [4]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ""

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import onnxruntime as rt
import tensorflow as tf
import tf2onnx
from transformers import BertTokenizer, TFBertForSequenceClassification, TFDistilBertForSequenceClassification
import tensorflow as tf
from tokenizers import BertWordPieceTokenizer

In [5]:
! wget https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt



--2021-12-06 17:05:01--  https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.160.144
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.160.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 231508 (226K) [text/plain]
Saving to: ‘bert-base-uncased-vocab.txt’

bert-base-uncased-v 100%[===================>] 226.08K  --.-KB/s    in 0.1s    

2021-12-06 17:05:01 (2.00 MB/s) - ‘bert-base-uncased-vocab.txt’ saved [231508/231508]



In [6]:
from transformers import DistilBertTokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=1)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/347M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_layer_norm', 'activation_13', 'vocab_projector', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'classifier', 'dropout_19']
You should probably TRAIN this model on a down-stream task to be able to use i

In [7]:
text = """
Voted Boston Globe\s 2018 best New England beach town! Narragansett is the perfect seaside community for a laid back coastal retreat. Immaculate, highly ranked by VRBO, 4+ bedroom three and a half bath Pottery Barn inspired shingle style beach home designed for families and entertaining. Main part of the house has four bedrooms two and a half baths with an outdoor shower and changing room. Additionally, there is a large, finished lower level GUEST SUITE with a kitchenette, fourth bathroom and queen sized bed in the sleeping area. Perfect for family getaways, reunions, anniversaries, graduations. and local wedding accommodations. Comfortable for as little as 4 to as many as 12.\n \n\nThe Harbour Island community is set on a peninsula that jets out into the salt pond. Enjoy barbecuing on our expanded deck leading onto one of the largest back yards on the island, or simply walk to kayaking, clamming, and paddle boarding on Point Judith.\n\nOur property is surrounded by four of Narragansett‚Äôs finest beaches just minutes away, and just up the street form a quaint private association beach/swim area (dock is for members only). Close to restaurants, shopping, Block Island ferry, and the center of town. Available primarily in the summer months weekly, monthly, or seasonally with advanced notice. \n\n*** HOUSE AMENITIES ***\n\nTwo Story Lofted Great Room, Central AC, WiFi, Custom Kitchen, Stainless Steel Appliances, Stunning Guest Suite With Private Bath and Kitchenette, Outdoor Dining Area, Large Yard, Gas Fire Pit With Seating, Weber Grill, Quality Pots and Pans, Keurig and Conventional Coffee, Mosquito and Tick Controlled, Stone Patio, Hammock, Front Porch, Multiple TV\s, Outdoor Shower/Changing Area, Town Beach Passes, Walk to Private Quaint Association Beach (Dock is For Members Only), Off Street Parking For Up To Six Cars, Walk To Paddle Boarding, Kayaking, Clamming and Sunsets\n\nImmaculate, 3,000 sq/ft Pottery Barn inspired beach house... Featured in the style section of "SO RI" magazine! Centrally located on Harbour Island. Four bedrooms with additional lower level Guest Suite for accommodating even more guests. Open floor plan for entertaining, custom kitchen, huge yard for grilling and relaxing, central AC, deck, loft, outdoor shower, stone patio, close to beaches, kayaking,paddle boarding, B. I. ferry, shopping, & restaurants.\n\nAbout the area...\nHarbour Island is centrally located in Narragansett. Our private Association Beach and swim area is just down the street from our front door (dock is for members only). It\s a great place to take a dip, catch some sun, or launch a kayak or paddle board. Narragansett Town beach passes provided...Sand Hill Cove, Galilee, Scarborough Beach, and the Block Island ferry are all within 2.5 to 4 miles of the property. Harbour Island is close to shops, restaurants, center of town, and market yet you feel like your in a quaint coastal setting. Newport, Providence, and Foxwoods casino are all short drives away.\nAbout the home...We have one of the largest yards on the island for grilling and relaxing with a new expanded patio and fire pit off our main deck so there is plenty of room for the whole gang! Home was built by owner to entertain with a wide open floor plan, lofted great room w/ 20 foot ceilings, top of the line kitchen, central air, porch, deck, custom built-ins, fire pit and distant water views from upper level. There is an emphasis on open, airy, light filled rooms.\nMain part of the home has four bedrooms two and one half baths, and a hot and cold outdoor shower with changing room. Additionally, there is a large family room with fifth queen bed, fourth bath, microwave and fridge in the lower level to accommodate even more guests. Comfortable for as little as 4 to as many as 12. We have many returning guests year after...We do our best to make your stay as comfortable as possible providing incidentals such as stainless cookware, blankets, comforters, pillows, beach towels, bath towels, lawn chairs, lawn games, trash bags, spare propane for Weber grill, laundry/dish detergent, etc. Fresh sheets and pillow cases arrive at your front door from Vacation Beach House Linens the day of your arrival. Smaller, hypoallergenic pets will be considered only after speaking with host.	
"""


In [8]:
%%timeit
predict_input = tokenizer.encode(text,
                                 truncation=True,
                                 padding=True,
                                 return_tensors="tf")
tf_output = model.predict(predict_input)[0]
# print("tf_output:  ", tf_output)
tf_prediction = tf.nn.softmax(tf_output, axis=1).numpy()[0]
# print("tf_prediction: ",  tf_prediction)

The slowest run took 5.38 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 5: 1.1 s per loop


## Convert to ONNX

In [9]:
# describe the inputs
input_spec = (
    tf.TensorSpec((None,  None), tf.int32, name="input_ids"),
)

# and convert
_, _ = tf2onnx.convert.from_keras(model, input_signature=input_spec, opset=13, output_path="distilbert.onnx")

Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


In [10]:
!du -hs /content/distilbert.onnx

256M	/content/distilbert.onnx


In [13]:
!pip install coloredlogs

     |████████████████████████████████| 46 kB 2.7 MB/s 
     |████████████████████████████████| 86 kB 2.9 MB/s 


In [22]:
# optimize transformer-based models with onnxruntime-tools
# from onnxruntime import optimizer
from onnxruntime.transformers.optimizer import optimize_model
from onnxruntime.transformers.onnx_model_bert import BertOptimizationOptions
from fusion_options import FusionOptions

# disable embedding layer norm optimization for better model size reduction
opt_options = FusionOptions('distilbert')
opt_options.enable_embed_layer_norm = False

opt_model = optimize_model(
    'distilbert.onnx',
    'bert', 
    # num_heads=12,
    # hidden_size=768, # these get detected automatically from graph: https://github.com/microsoft/onnxruntime/blob/5e197698b8964875bb9b979070c325b51bb0228a/onnxruntime/python/tools/transformers/optimizer.py#L184
    optimization_options=opt_options)
opt_model.save_model_to_file('distilbert.opt.onnx')


Model producer not matched: Expect pytorch, Got tf2onnx 1.9.3. Please specify correct --model_type parameter.


In [23]:
!du -hs distilbert.opt.onnx

256M	distilbert.opt.onnx


# **Dynamic quantization**

In [30]:
def quantize_onnx_model(onnx_model_path, quantized_model_path):
    from onnxruntime.quantization import quantize_dynamic, QuantType
    import onnx
    onnx_opt_model = onnx.load(onnx_model_path)
    quantize_dynamic(onnx_model_path,
                     quantized_model_path,
                     weight_type=QuantType.QInt8)

    print(f"quantized model saved to:{quantized_model_path}")

quantize_onnx_model('distilbert.opt.onnx', 'distilbert.opt.quant.onnx')

print('ONNX full precision model size (MB):', os.path.getsize("distilbert.opt.onnx")/(1024*1024))
print('ONNX quantized model size (MB):', os.path.getsize("distilbert.opt.quant.onnx")/(1024*1024))

quantized model saved to:distilbert.opt.quant.onnx
ONNX full precision model size (MB): 255.6575698852539
ONNX quantized model size (MB): 64.46626663208008


In [32]:
!du -hs distilbert.opt.quant.onnx

65M	distilbert.opt.quant.onnx


## Test the ONNX model with onnxruntime

In [24]:
opt = rt.SessionOptions()
ort_session = rt.InferenceSession("distilbert.opt.onnx")

In [27]:
%%timeit
tokenized_text = tokenizer.encode(text,
                                 truncation=True,
                                 padding=True,
                                 return_tensors="tf")
input_ids=tokenized_text
input_dict = {"input_ids" : input_ids.numpy() }
onnx_results = ort_session.run(None, input_dict)
# onnx_results

1 loop, best of 5: 840 ms per loop


## Test the Quantized ONNX model with onnxruntime

In [35]:
opt = rt.SessionOptions()
ort_session = rt.InferenceSession("distilbert.opt.quant.onnx")

In [36]:
%%timeit
tokenized_text = tokenizer.encode(text,
                                 truncation=True,
                                 padding=True,
                                 return_tensors="tf")
input_ids=tokenized_text
input_dict = {"input_ids" : input_ids.numpy() }
onnx_results = ort_session.run(None, input_dict)
# onnx_results

1 loop, best of 5: 845 ms per loop


## Make sure tensorflow and onnxruntime results are the same

In [183]:
# outputs are matching
np.testing.assert_allclose(tf_results['logits'], onnx_results[0], rtol=1e-5, atol=1e-5)

AssertionError: ignored